In [1]:

import json, random
from torch.utils.data import Dataset
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
from torchvision import models
import numpy as np


/home/xingxing/anaconda3/envs/vitscn/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pinyin_list = ['a', 'ai', 'an', 'ang', 'ao', 'ba', 'bai', 'ban', 'bang', 'bao', 'be', 'bei', 'ben', 'beng', 'bi', 'bian', 'bianr', 'biao', 'bie', 'bin', 'bing', 'bo', 'bu', 'ca', 'cai', 'can', 'cang', 'cao', 'ce', 'cen', 'ceng', 'cha', 'chai', 'chan', 'chang', 'chao', 'che', 'chen', 'cheng', 'chi', 'chong', 'chou', 'chu', 'chua', 'chuai', 'chuan', 'chuang', 'chui', 'chun', 'chuo', 'ci', 'cong', 'cou', 'cu', 'cuan', 'cui', 'cun', 'cuo', 'da', 'dai', 'dan', 'dang', 'dao', 'de', 'dei', 'den', 'deng', 'di', 'dia', 'dian', 'diao', 'die', 'ding', 'diu', 'dong', 'dou', 'du', 'duan', 'dui', 'dun', 'duo', 'e', 'ei', 'en', 'ng', 'eng', 'er', 'fa', 'fan', 'fang', 'fei', 'fen', 'feng', 'fo', 'fou', 'fu', 'ga', 'gai', 'gan', 'gang', 'gao', 'ge', 'gei', 'gen', 'geng', 'gong', 'gou', 'gu', 'gua', 'guai', 'guan', 'guang', 'gui', 'gun', 'guo', 'ha', 'hai', 'hair', 'han', 'hang', 'hao', 'he', 'hei', 'hen', 'heng', 'hong', 'hou', 'hu', 'hua', 'huai', 'huan', 'huang', 'hui', 'huir', 'hun', 'huo', 'ji', 'jia', 'jian', 'jiang', 'jiao', 'jie', 'jin', 'jing', 'jiong', 'jiu', 'ju', 'juan', 'jue', 'jun', 'ka', 'kai', 'kan', 'kang', 'kao', 'ke', 'kei', 'ken', 'keng', 'kong', 'kou', 'ku', 'kua', 'kuai', 'kuan', 'kuang', 'kui', 'kun', 'kuo', 'la', 'lai', 'lan', 'lang', 'lao', 'le', 'lei', 'leng', 'li', 'lia', 'lian', 'liang', 'liao', 'lie', 'lin', 'ling', 'liu', 'lo', 'long', 'lou', 'lu', 'lv', 'luan', 'lve', 'lue', 'lun', 'luo', 'ma', 'mai', 'man', 'mang', 'mao', 'me', 'mei', 'men', 'meng', 'mi', 'mian', 'miao', 'mie', 'min', 'ming', 'miu', 'mo', 'mou', 'mu', 'na', 'nar', 'nai', 'nan', 'nang', 'nao', 'ne', 'nei', 'nen', 'neng', 'ni', 'nia', 'nian', 'niang', 'niao', 'nie', 'nin', 'ning', 'niu', 'nong', 'nou', 'nu', 'nv', 'nuan', 'nve', 'nue', 'nuo', 'o', 'ou', 'pa', 'pai', 'pan', 'pang', 'pao', 'pe', 'pei', 'pen', 'peng', 'pi', 'pian', 'piao', 'pie', 'pin', 'ping', 'po', 'pou', 'pu', 'qi', 'qia', 'qian', 'qiang', 'qiao', 'qie', 'qin', 'qing', 'qiong', 'qiu', 'qu', 'quan', 'que', 'qun', 'ran', 'rang', 'rao', 're', 'ren', 'reng', 'ri', 'rong', 'rou', 'ru', 'rua', 'ruan', 'rui', 'run', 'ruo', 'sa', 'sai', 'san', 'sang', 'sao', 'se', 'sen', 'seng', 'sha', 'shai', 'shan', 'shang', 'shao', 'she', 'shei', 'shen', 'sheng', 'shi', 'shir', 'shou', 'shu', 'shua', 'shuai', 'shuan', 'shuang', 'shui', 'shun', 'shuo', 'si', 'song', 'sou', 'su', 'suan', 'sui', 'sun', 'suo', 'ta', 'tai', 'tan', 'tang', 'tao', 'te', 'tei', 'teng', 'ti', 'tian', 'tianr', 'tiao', 'tie', 'ting', 'tong', 'tou', 'tu', 'tuan', 'tui', 'tun', 'tuo', 'wa', 'wai', 'wan', 'wanr', 'wang', 'wei', 'weir', 'wen', 'weng', 'wo', 'wu', 'xi', 'xia', 'xian', 'xiang', 'xiao', 'xie', 'xin', 'xing', 'xiong', 'xiu', 'xu', 'xuan', 'xue', 'xun', 'ya', 'yan', 'yang', 'yao', 'ye', 'yi', 'yin', 'ying', 'yo', 'yong', 'you', 'yu', 'yuan', 'yue', 'yun', 'za', 'zai', 'zan', 'zang', 'zao', 'ze', 'zei', 'zen', 'zeng', 'zha', 'zhai', 'zhan', 'zhang', 'zhao', 'zhe', 'zhei', 'zhen', 'zheng', 'zher', 'zhi', 'zhong', 'zhou', 'zhu', 'zhua', 'zhuai', 'zhuan', 'zhuang', 'zhui', 'zhun', 'zhuo', 'zi', 'zong', 'zou', 'zu', 'zuan', 'zui', 'zun', 'zuo', 'gair', 'yir', 'jinr', 'menr', 'her', 'gunr', 'huanr', 'jiar', 'duir', 'dir', 'banr', 'yanr', 'cir', 'genr', 'ger', 'tongr', 'renr', 'kuair', 'yuanr', 'liar', 'dunr', 'fur', 'zaor', 'yangr', 'ter', 'yingr', 'fanr', 'tuor', 'pir', 'jingr', 'hanr', 'duor', 'tanr', 'zhunr', 'jir', 'huor', 'far', 'dianr', 'zhur', 'tour', 'qir', 'mingr', 'lanr', 'lir', 'daor', 'niur']

pinyin_to_label = {p: i for i, p in enumerate(pinyin_list)}
label_to_pinyin = {i: p for i, p in enumerate(pinyin_list)}

In [3]:
class PinyinDataset(Dataset):
    def __init__(self, audio_py_pairs):
        self.audio_py_pairs = audio_py_pairs
        
    def __len__(self):
        return len(self.audio_py_pairs)
    
    def get_audio_py_pair(self, pair):
        file_path, py, start, end, f_npy = pair
        
        stacked = torch.from_numpy(np.load(open(f_npy, 'rb')))
        
        py_wo_tone = py[:-1]
        label = pinyin_to_label[py_wo_tone]
                
        return stacked, label, file_path, start, end, py
        
    def __getitem__(self, idx):
        return self.get_audio_py_pair(self.audio_py_pairs[idx])

In [4]:
audio_py_pairs = json.loads(open('f_audio_py_pairs_thubert_aug.json', "r").read())
random.seed(1)
random.shuffle(audio_py_pairs)

train_pairs, test_pairs = audio_py_pairs[:int(len(audio_py_pairs)*.9)], audio_py_pairs[int(len(audio_py_pairs)*.9):]

train_data = PinyinDataset(train_pairs)
test_data = PinyinDataset(test_pairs)

tr = set([tp[1][:-1] for tp in train_pairs])
te = set([tp[1][:-1] for tp in test_pairs])
print(len(tr), len(te), len(audio_py_pairs))
te - tr

train_loader = DataLoader(dataset=train_data, num_workers=8, batch_size=128, shuffle=False)
test_loader = DataLoader(dataset=test_data, num_workers=8, batch_size=128, shuffle=False)


457 456 130447


In [5]:
def change_layers(model):
    model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    model.fc = nn.Linear(512, len(pinyin_list))
        
    return model

model = models.resnet18(pretrained=False)
change_layers(model)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [10]:
model.load_state_dict(torch.load('checkpoints/13_09_2023_22:13:20/ckp_12.pt')['model_state_dict'])
model = model.cuda()

In [11]:
model.eval()
model = model.cuda()

for k in [1, 2, 3, 5]:
    correct = 0
    total = 0    
    for inputs, labels, file_path, start, end, py in test_loader:
        inputs, labels = inputs.cuda(), labels.cuda()
        
        with torch.no_grad():
            output = model(inputs)
        _, predicted = torch.topk(output, k)
        correct += sum([1 if labels[i] in predicted[i] else 0 for i in range(len(predicted))])
        for i in range(len(predicted)):
            if labels[i] not in predicted[i]:
                pass
                # print(label_to_pinyin[predicted[i][0].item()], label_to_pinyin[labels[i].item()])
                # print(file_path[i], start[i].item(), end[i].item(), py[i])
        total += labels.size(0)

    print('k: %d   Accuracy of the model: %.3f %%' %(k, (100*correct)/(total)))

k: 1   Accuracy of the model: 96.727 %
k: 2   Accuracy of the model: 99.310 %
k: 3   Accuracy of the model: 99.663 %
k: 5   Accuracy of the model: 99.854 %
